# SAE Feature-Attribution Correlation Analysis

This notebook:
1. Loads cell data with geneformer embeddings from CellxGene Census
2. Passes embeddings through the trained Sparse Autoencoder
3. Extracts SAE latent features
4. Analyzes correlations between SAE features and cell attributions

## Setup and Imports

In [7]:
import sys
import os
import json
import pandas as pd
import numpy as np
import torch
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns
import cellxgene_census
from cellxgene_census.experimental import get_embedding, get_embedding_metadata, get_all_available_embeddings
from scipy.sparse import issparse

# GSEAPY for pathway analysis
import gseapy as gp

# Local imports
sys.path.append('../src')
from models import SparseAutoencoder
from feature_attribution_analysis import FeatureAttributionAnalyzer

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Configuration
ORGANISM = "homo_sapiens"
MEASUREMENT = "RNA"
CENSUS_VERSION = "2025-01-30"
EMBEDDING_NAME = "geneformer"
SAMPLE_SIZE = 5000  # Number of cells to analyze (reduced for comprehensive annotations)

# Metadata fields to collect (comprehensive set)
METADATA_FIELDS = [
    "assay",
    "dataset_id",
    "cell_type",
    "development_stage",
    "disease",
    "self_reported_ethnicity",
    "sex",
    "tissue_general",
    "tissue",
    "soma_joinid"  # Need this for joining with expression data
]

## Step 1: Load Data from CellxGene Census

Get normal cells with geneformer embeddings and metadata

In [6]:
def get_census_data_with_embeddings(sample_size=SAMPLE_SIZE, census_version=CENSUS_VERSION):
    """
    Retrieve cell data with embeddings and comprehensive metadata from CellxGene Census.
    Uses the same approach as the correlation set notebook for consistency.
    """
    print(f"Fetching {sample_size} cells with {EMBEDDING_NAME} embeddings and comprehensive metadata...")

    with cellxgene_census.open_soma(census_version=census_version) as census:
        adata = cellxgene_census.get_anndata(
            census,
            organism=ORGANISM,
            measurement_name=MEASUREMENT,
            obs_value_filter=f"soma_joinid < {sample_size}",  # Simple filtering approach
            var_value_filter="feature_type=='protein_coding'",
            obs_embeddings=[EMBEDDING_NAME],
            obs_column_names=METADATA_FIELDS,  # Explicit metadata collection
        )

        print(f"Retrieved {adata.shape[0]} cells x {adata.shape[1]} genes")

        # Set feature names properly
        adata.var_names = adata.var["feature_name"]

    return adata

# Load the data
adata = get_census_data_with_embeddings()
print(f"Final dataset: {adata.shape[0]} cells x {adata.shape[1]} genes")
print(f"Available embeddings: {list(adata.obsm.keys())}")
print(f"Geneformer embedding shape: {adata.obsm[EMBEDDING_NAME].shape}")
print(f"Metadata fields: {METADATA_FIELDS}")
print(f"Unique cell types: {adata.obs['cell_type'].nunique()}")
print(f"Unique tissues: {adata.obs['tissue_general'].nunique()}")
print(f"Disease distribution: {adata.obs['disease'].value_counts().head()}")

Fetching 5000 cells with geneformer embeddings and comprehensive metadata...


/nfs/homes/bargho16/.local/share/uv/python/cpython-3.13.9-linux-x86_64-gnu/lib/python3.13/functools.py:934: ImplicitModificationWarning: Transforming to str index.
  return dispatch(args[0].__class__)(*args, **kw)
/nfs/homes/bargho16/.local/share/uv/python/cpython-3.13.9-linux-x86_64-gnu/lib/python3.13/functools.py:934: ImplicitModificationWarning: Transforming to str index.
  return dispatch(args[0].__class__)(*args, **kw)


Retrieved 5000 cells x 20045 genes
Final dataset: 5000 cells x 20045 genes
Available embeddings: ['geneformer']
Geneformer embedding shape: (5000, 512)
Metadata fields: ['assay', 'dataset_id', 'cell_type', 'development_stage', 'disease', 'self_reported_ethnicity', 'sex', 'tissue_general', 'tissue', 'soma_joinid']
Unique cell types: 20
Unique tissues: 4
Disease distribution: disease
Alzheimer disease              2442
normal                         1993
breast cancer                   565
Barrett esophagus                 0
B-cell non-Hodgkin lymphoma       0
Name: count, dtype: int64
Final dataset: 5000 cells x 20045 genes
Available embeddings: ['geneformer']
Geneformer embedding shape: (5000, 512)
Metadata fields: ['assay', 'dataset_id', 'cell_type', 'development_stage', 'disease', 'self_reported_ethnicity', 'sex', 'tissue_general', 'tissue', 'soma_joinid']
Unique cell types: 20
Unique tissues: 4
Disease distribution: disease
Alzheimer disease              2442
normal                 

## Step 2: Load Trained Sparse Autoencoder

Load the pre-trained SAE model and hyperparameters

In [9]:
# Load model hyperparameters
params_path = "../models/best_sparse_autoencoder_params.json"
with open(params_path, "r") as f:
    params = json.load(f)

print("SAE Hyperparameters:")
for key, value in params.items():
    print(f"  {key}: {value}")

# Initialize the model with saved hyperparameters
sae_model = SparseAutoencoder(
    input_dim=512,  # Geneformer embedding dimension
    hidden_dim=params["hidden_dim"],
    expanded_ratio=params["expanded_ratio"],
    n_encoder_layers=params["n_encoder_layers"],
    n_decoder_layers=params["n_decoder_layers"]
)

# Load trained weights
checkpoint_path = "../models/best_sparse_autoencoder.pt"
sae_model.load_state_dict(torch.load(checkpoint_path, map_location='cpu'))
sae_model.eval()

print(f"\nLoaded SAE model from: {checkpoint_path}")
print(f"SAE expanded dimension: {sae_model.expanded_dim}")
print(f"SAE hidden dimension: {sae_model.hidden_dim}")
print(f"SAE input dimension: {sae_model.input_dim}")

SAE Hyperparameters:
  hidden_dim: 512
  expanded_ratio: 10.0
  n_encoder_layers: 2
  n_decoder_layers: 1
  lr: 0.00017701853566940679
  sparsity_weight: 0.0006659703893931954

Loaded SAE model from: ../models/best_sparse_autoencoder.pt
SAE expanded dimension: 5120
SAE hidden dimension: 512
SAE input dimension: 512


## Step 3: Extract SAE Latent Features

Pass geneformer embeddings through the SAE to get latent representations

In [22]:
def extract_sae_features(embeddings, model):
    """
    Extract SAE latent features from input embeddings.
    Uses the full forward pass to get the proper sparse features.
    """
    model.eval()

    with torch.no_grad():
        # Convert to tensor
        embeddings_tensor = torch.FloatTensor(embeddings)

        # Use the full forward pass to get the sparse features
        # The forward method returns (reconstructed, features) where features is the sparse representation
        reconstructed, sparse_features = model(embeddings_tensor)

    return sparse_features.numpy()

# Extract geneformer embeddings
geneformer_embeddings = adata.obsm[EMBEDDING_NAME]
print(f"Geneformer embeddings shape: {geneformer_embeddings.shape}")

# Extract SAE latent features using the corrected method
print("Extracting SAE sparse features using full forward pass...")
sae_features = extract_sae_features(geneformer_embeddings, sae_model)
print(f"SAE features shape: {sae_features.shape}")

# Check sparsity
sparsity = (sae_features == 0).mean()
print(f"SAE feature sparsity: {sparsity:.2%}")

# Add SAE features to adata for convenience
adata.obsm['sae_features'] = sae_features

Geneformer embeddings shape: (5000, 512)
Extracting SAE sparse features using full forward pass...
SAE features shape: (5000, 5120)
SAE feature sparsity: 39.96%


## Step 4: Prepare Comprehensive Cell Attributions

Extract and compute comprehensive biological annotations including:
- Technical metrics (library size, gene counts, mitochondrial/ribosomal content)
- Cell cycle scoring
- Pathway activity via ssGSEA (inflammation, hypoxia)
- Existing metadata (cell type, tissue, disease, etc.)

In [12]:
# Initialize comprehensive annotations dataframe
annotations_df = pd.DataFrame(index=adata.obs_names)

# Add existing metadata fields
for field in METADATA_FIELDS:
    if field in adata.obs.columns:
        annotations_df[field] = adata.obs[field]

print("Added metadata fields:")
for field in METADATA_FIELDS:
    if field in adata.obs.columns:
        print(f"  {field}: {adata.obs[field].nunique()} unique values")

# Examine what we have so far
print(f"\nCurrent annotations shape: {annotations_df.shape}")
print("Sample of existing metadata:")
print(annotations_df.head())

Added metadata fields:
  assay: 4 unique values
  dataset_id: 5 unique values
  cell_type: 20 unique values
  development_stage: 27 unique values
  disease: 3 unique values
  self_reported_ethnicity: 2 unique values
  sex: 3 unique values
  tissue_general: 4 unique values
  tissue: 5 unique values
  soma_joinid: 5000 unique values

Current annotations shape: (5000, 10)
Sample of existing metadata:
       assay                            dataset_id         cell_type  \
0  10x 3' v3  d7476ae2-e320-4703-8304-da5c42627e71  endothelial cell   
1  10x 3' v3  d7476ae2-e320-4703-8304-da5c42627e71    malignant cell   
2  10x 3' v3  d7476ae2-e320-4703-8304-da5c42627e71        fibroblast   
3  10x 3' v3  d7476ae2-e320-4703-8304-da5c42627e71        fibroblast   
4  10x 3' v3  d7476ae2-e320-4703-8304-da5c42627e71        macrophage   

   development_stage        disease self_reported_ethnicity     sex  \
0  29-year-old stage  breast cancer                European  female   
1  29-year-old stage  br

In [13]:
# Compute technical metrics
print("Computing technical QC metrics...")

# Get expression matrix
if issparse(adata.X):
    X = adata.X.toarray()
else:
    X = adata.X

# Basic technical metrics
annotations_df['n_counts'] = X.sum(axis=1)
annotations_df['n_genes'] = (X > 0).sum(axis=1)

# Mitochondrial gene percentage
mito_genes = adata.var_names.str.upper().str.startswith("MT-")
annotations_df['pct_mito'] = X[:, mito_genes].sum(axis=1) / annotations_df['n_counts'] * 100

# Ribosomal gene percentage
ribo_genes = adata.var_names.str.startswith(("RPS","RPL"))
annotations_df['pct_ribo'] = X[:, ribo_genes].sum(axis=1) / annotations_df['n_counts'] * 100

print(f"Technical metrics computed:")
print(f"  Mean counts per cell: {annotations_df['n_counts'].mean():.0f}")
print(f"  Mean genes per cell: {annotations_df['n_genes'].mean():.0f}")
print(f"  Mean % mitochondrial: {annotations_df['pct_mito'].mean():.1f}%")
print(f"  Mean % ribosomal: {annotations_df['pct_ribo'].mean():.1f}%")

Computing technical QC metrics...
Technical metrics computed:
  Mean counts per cell: 2032
  Mean genes per cell: 738
  Mean % mitochondrial: 4.8%
  Mean % ribosomal: 9.0%
Technical metrics computed:
  Mean counts per cell: 2032
  Mean genes per cell: 738
  Mean % mitochondrial: 4.8%
  Mean % ribosomal: 9.0%


In [14]:
# Get Hallmark gene sets for cell cycle and pathway analysis
print("Loading Hallmark gene sets...")
hallmark_genesets = gp.get_library(name='MSigDB_Hallmark_2020', organism='Human')

# Cell cycle scoring
print("Computing cell cycle scores...")
s_genes = [g for g in hallmark_genesets['E2F Targets'] if g in adata.var_names]
g2m_genes = [g for g in hallmark_genesets['G2-M Checkpoint'] if g in adata.var_names]

print(f"Found {len(s_genes)} S-phase genes and {len(g2m_genes)} G2/M genes in dataset")

if len(s_genes) > 0 and len(g2m_genes) > 0:
    sc.tl.score_genes_cell_cycle(adata, s_genes=s_genes, g2m_genes=g2m_genes, copy=False)
    annotations_df['S_score'] = adata.obs['S_score']
    annotations_df['G2M_score'] = adata.obs['G2M_score']
    annotations_df['phase'] = adata.obs['phase']
    print(f"Cell cycle phases: {adata.obs['phase'].value_counts().to_dict()}")
else:
    print("Warning: Insufficient cell cycle genes found, skipping cell cycle scoring")

Loading Hallmark gene sets...
Computing cell cycle scores...
Found 200 S-phase genes and 199 G2/M genes in dataset
Computing cell cycle scores...
Found 200 S-phase genes and 199 G2/M genes in dataset
Cell cycle phases: {'G1': 2655, 'G2M': 1845, 'S': 500}
Cell cycle phases: {'G1': 2655, 'G2M': 1845, 'S': 500}


In [15]:
# Pathway activity analysis via ssGSEA
print("Computing pathway activity scores...")

# Define key biological pathways to analyze
marker_sets = {
    "Inflammation_Response": hallmark_genesets['Inflammatory Response'],
    "Hypoxia": hallmark_genesets['Hypoxia'],
    "Apoptosis": hallmark_genesets['Apoptosis'],
    "DNA_Repair": hallmark_genesets['DNA Repair'],
    "Oxidative_Phosphorylation": hallmark_genesets['Oxidative Phosphorylation']
}

# Prepare expression data for ssGSEA
expr_df = pd.DataFrame(X.T, index=adata.var_names, columns=adata.obs_names)

# Compute ssGSEA scores for each pathway
for pathway_name, genes in marker_sets.items():
    genes_present = [g for g in genes if g in adata.var_names]
    if len(genes_present) == 0:
        print(f"Warning: no genes from {pathway_name} found in dataset, skipping")
        continue

    print(f"Computing {pathway_name} score ({len(genes_present)} genes)...")

    try:
        ss_res = gp.ssgsea(
            data=expr_df,
            gene_sets={pathway_name: genes_present},
            sample_norm_method="rank",
            outdir=None,
            no_plot=True,
            permutation_num=0
        )

        # Extract scores and add to annotations
        sample_names = ss_res.res2d['Name']
        if sample_names.dtype != expr_df.columns.dtype:
            sample_names = sample_names.astype(expr_df.columns.dtype)
        nes_series = pd.Series(data=ss_res.res2d['NES'].values, index=sample_names)

        annotations_df[pathway_name] = annotations_df.index.map(lambda i: nes_series[i])
        print(f"  {pathway_name}: mean score = {annotations_df[pathway_name].mean():.3f}")

    except Exception as e:
        print(f"Error computing {pathway_name}: {e}")
        continue

Computing pathway activity scores...
Computing Inflammation_Response score (199 genes)...
  Inflammation_Response: mean score = -0.299
Computing Hypoxia score (199 genes)...
  Inflammation_Response: mean score = -0.299
Computing Hypoxia score (199 genes)...
  Hypoxia: mean score = -0.510
Computing Apoptosis score (160 genes)...
  Hypoxia: mean score = -0.510
Computing Apoptosis score (160 genes)...
  Apoptosis: mean score = -0.390
Computing DNA_Repair score (149 genes)...
  Apoptosis: mean score = -0.390
Computing DNA_Repair score (149 genes)...
  DNA_Repair: mean score = -0.297
Computing Oxidative_Phosphorylation score (199 genes)...
  DNA_Repair: mean score = -0.297
Computing Oxidative_Phosphorylation score (199 genes)...
  Oxidative_Phosphorylation: mean score = -0.222
  Oxidative_Phosphorylation: mean score = -0.222


In [16]:
# Add all annotations to adata.obs
print("Integrating annotations into AnnData object...")
for col in annotations_df.columns:
    adata.obs[col] = annotations_df[col]

print(f"Final annotations shape: {annotations_df.shape}")
print("Summary of all annotations:")
print(annotations_df.describe())

# Display comprehensive annotation summary
print("\n=== COMPREHENSIVE ANNOTATION SUMMARY ===")
print(f"Total cells: {adata.shape[0]}")
print(f"Total annotations: {annotations_df.shape[1]}")

print(f"\nMetadata fields:")
for field in METADATA_FIELDS:
    if field in adata.obs.columns:
        n_unique = adata.obs[field].nunique()
        print(f"  {field}: {n_unique} unique values")

print(f"\nTechnical metrics:")
tech_metrics = ['n_counts', 'n_genes', 'pct_mito', 'pct_ribo']
for metric in tech_metrics:
    if metric in adata.obs.columns:
        mean_val = adata.obs[metric].mean()
        std_val = adata.obs[metric].std()
        print(f"  {metric}: {mean_val:.2f} ± {std_val:.2f}")

print(f"\nCell cycle:")
if 'phase' in adata.obs.columns:
    print(f"  Phase distribution: {adata.obs['phase'].value_counts().to_dict()}")

print(f"\nPathway scores:")
pathway_cols = [col for col in adata.obs.columns if any(x in col for x in ['Inflammation', 'Hypoxia', 'Apoptosis', 'DNA_Repair', 'Oxidative'])]
for col in pathway_cols:
    mean_val = adata.obs[col].mean()
    print(f"  {col}: {mean_val:.3f}")

Integrating annotations into AnnData object...
Final annotations shape: (5000, 22)
Summary of all annotations:
       soma_joinid      n_counts      n_genes     pct_mito     pct_ribo  \
count  5000.000000   5000.000000  5000.000000  5000.000000  5000.000000   
mean   2499.500000   2031.623047   737.590200     4.811214     8.978488   
std    1443.520003   3955.448242   728.876894    10.417015    12.041237   
min       0.000000    135.000000   101.000000     0.000000     0.000000   
25%    1249.750000    368.000000   307.750000     0.242718     1.363636   
50%    2499.500000    642.000000   476.500000     0.807538     2.393635   
75%    3749.250000   1558.000000   864.250000     2.826175    12.871259   
max    4999.000000  52427.000000  6961.000000    71.299095    63.765736   

           S_score    G2M_score  Inflammation_Response      Hypoxia  \
count  5000.000000  5000.000000            5000.000000  5000.000000   
mean     -0.005324     0.016416              -0.298554    -0.509694   


## Step 5: Run Feature-Attribution Analysis

Analyze correlations between SAE features and cell attributions using the comprehensive statistical toolkit

In [ ]:
# Smart categorization and preparation of attributions for analysis
print("Categorizing and preparing attributions for analysis...")

# Smart categorization based on biological knowledge
binary_attrs = []
categorical_attrs = []
continuous_attrs = []

# Predefined categories based on biological knowledge
known_categorical = [
    'cell_type', 'tissue_general', 'tissue', 'disease', 'development_stage',
    'assay', 'dataset_id', 'self_reported_ethnicity', 'phase'
]
known_continuous = [
    'n_counts', 'n_genes', 'pct_mito', 'pct_ribo', 'S_score', 'G2M_score'
] + [col for col in adata.obs.columns if any(x in col for x in ['Inflammation', 'Hypoxia', 'Apoptosis', 'DNA_Repair', 'Oxidative'])]

# Categorize based on data properties and biological knowledge
for col in adata.obs.columns:
    if col.startswith('_') or col == 'soma_joinid':  # Skip internal columns
        continue

    n_unique = adata.obs[col].nunique()
    dtype = adata.obs[col].dtype
    missing_frac = adata.obs[col].isna().mean()

    # Skip if too many missing values
    if missing_frac > 0.5:
        continue

    # Categorize based on biological knowledge and data properties
    if col in known_continuous:
        continuous_attrs.append(col)
    elif col in known_categorical:
        if n_unique <= 50:  # Reasonable number of categories
            categorical_attrs.append(col)
    elif n_unique == 2 and dtype == 'object':
        binary_attrs.append(col)
    elif 2 < n_unique <= 50 and dtype == 'object':
        categorical_attrs.append(col)
    elif dtype in ['float64', 'int64'] and n_unique > 10:
        continuous_attrs.append(col)

print(f"Categorized attributes:")
print(f"  Binary: {len(binary_attrs)} attributes - {binary_attrs}")
print(f"  Categorical: {len(categorical_attrs)} attributes - {categorical_attrs}")
print(f"  Continuous: {len(continuous_attrs)} attributes - {continuous_attrs}")

# Create attribution dictionary
attributions = {
    'binary': {attr: adata.obs[attr] for attr in binary_attrs},
    'categorical': {attr: adata.obs[attr] for attr in categorical_attrs},
    'continuous': {attr: adata.obs[attr] for attr in continuous_attrs}
}

# Create confounders for residualization (technical factors only - keep numeric)
confounder_cols = ['n_counts', 'n_genes']
confounders = adata.obs[confounder_cols].copy()

# Add a few top dataset dummies if we have multiple datasets, but limit to avoid overfitting
if 'dataset_id' in adata.obs.columns:
    top_datasets = adata.obs['dataset_id'].value_counts().head(10).index  # Top 10 datasets
    for dataset in top_datasets:
        confounders[f'dataset_{dataset}'] = (adata.obs['dataset_id'] == dataset).astype(int)

# Ensure all confounders are numeric
confounders = confounders.select_dtypes(include=[np.number])

print(f"\nConfounders shape: {confounders.shape}")
print(f"Confounder columns: {list(confounders.columns)}")
print(f"Total attributions: {sum(len(v) for v in attributions.values())}")
print(f"Ready for analysis!")

Categorizing and preparing attributions for analysis...
Categorized attributes:
  Binary: 0 attributes - []
  Categorical: 9 attributes - ['assay', 'dataset_id', 'cell_type', 'development_stage', 'disease', 'self_reported_ethnicity', 'tissue_general', 'tissue', 'phase']
  Continuous: 11 attributes - ['S_score', 'G2M_score', 'n_counts', 'n_genes', 'pct_mito', 'pct_ribo', 'Inflammation_Response', 'Hypoxia', 'Apoptosis', 'DNA_Repair', 'Oxidative_Phosphorylation']

Confounders shape: (5000, 12)
Confounder columns: ['n_counts', 'n_genes', 'dataset_bdacc907-7c26-419f-8808-969eab3ca2e8', 'dataset_fbd69faa-b0c5-45ba-89c9-da938a7f5a14', 'dataset_d7476ae2-e320-4703-8304-da5c42627e71', 'dataset_00ff600e-6e2e-4d76-846f-0eec4f0ae417', 'dataset_0895c838-e550-48a3-a777-dbcd35d30272', 'dataset_ae45e70d-cae7-45f5-8ee8-6655f208273c', 'dataset_ae4f8ddd-cac9-4172-9681-2175da462f2e', 'dataset_ae5341b8-60fb-4fac-86db-86e49ee66287', 'dataset_af8b241a-c72c-4470-b1a4-80e7336c6ab6', 'dataset_b03e4ef8-4e6b-47f4-

In [21]:
# Initialize the analyzer
analyzer = FeatureAttributionAnalyzer(
    n_permutations=1000,
    fdr_method='fdr_bh',
    cv_folds=5,
    random_state=42
)

print("Running comprehensive feature-attribution analysis...")
print(f"Analyzing {sae_features.shape[1]} SAE features vs {sum(len(v) for v in attributions.values())} attributions")

# Run the full analysis
results = analyzer.analyze_associations(
    features=sae_features,
    attributions=attributions,
    confounders=confounders.values if confounders.shape[1] > 0 else None
)

print("\nAnalysis completed!")

Running comprehensive feature-attribution analysis...
Analyzing 512 SAE features vs 20 attributions


/nfs/homes/bargho16/AstroBio/notebooks/../src/feature_attribution_analysis.py:409: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  r_s, p_s = spearmanr(feature_vals, attr_values, nan_policy='omit')


KeyboardInterrupt: 

## Step 6: Examine Results

### Univariate Association Results

In [ ]:
# Display summary statistics
print("=== ANALYSIS SUMMARY ===")
univar = results['univariate']

for attr_type in ['binary', 'categorical', 'continuous']:
    if attr_type in univar and len(univar[attr_type]) > 0:
        print(f"\n{attr_type.upper()} ATTRIBUTIONS:")

        for attr_name, attr_results in univar[attr_type].items():
            if len(attr_results) > 0:
                # Count significant associations
                sig_count = sum(1 for result in attr_results if result.get('q_value', 1.0) < 0.05)
                total_count = len(attr_results)

                print(f"  {attr_name}: {sig_count}/{total_count} significant SAE features (q < 0.05)")

                # Show top associations
                if sig_count > 0:
                    sorted_results = sorted(attr_results, key=lambda x: x.get('q_value', 1.0))
                    top_result = sorted_results[0]
                    print(f"    Best: Feature {top_result['feature_idx']}, q={top_result['q_value']:.2e}, effect={top_result.get('effect_size', 'N/A')}")

### Visualization: Association Heatmaps

In [ ]:
def create_association_heatmap(results, attr_type, metric='q_value', top_n=50):
    """
    Create heatmap of associations between SAE features and attributions.
    """
    if attr_type not in results['univariate'] or len(results['univariate'][attr_type]) == 0:
        print(f"No {attr_type} results to plot")
        return

    # Collect data for heatmap
    heatmap_data = []

    for attr_name, attr_results in results['univariate'][attr_type].items():
        for result in attr_results:
            heatmap_data.append({
                'attribution': attr_name,
                'feature': f"SAE_{result['feature_idx']}",
                'q_value': result.get('q_value', 1.0),
                'effect_size': result.get('effect_size', 0),
                'statistic': result.get('statistic', 0)
            })

    if len(heatmap_data) == 0:
        print(f"No data for {attr_type} heatmap")
        return

    df = pd.DataFrame(heatmap_data)

    # Filter to top associations
    df_sig = df[df['q_value'] < 0.05]

    if len(df_sig) == 0:
        print(f"No significant associations for {attr_type}")
        return

    # Take top N by effect size or q-value
    df_top = df_sig.nsmallest(min(top_n, len(df_sig)), 'q_value')

    # Create pivot table
    if metric == 'q_value':
        values = -np.log10(df_top['q_value'])
        label = '-log10(q-value)'
    else:
        values = df_top['effect_size']
        label = 'Effect Size'

    pivot_df = df_top.pivot_table(
        index='feature',
        columns='attribution',
        values=metric,
        fill_value=1.0 if metric == 'q_value' else 0.0
    )

    if metric == 'q_value':
        pivot_df = -np.log10(pivot_df)

    # Plot
    plt.figure(figsize=(12, 8))
    sns.heatmap(pivot_df,
                cmap='viridis',
                cbar_kws={'label': label},
                xticklabels=True,
                yticklabels=True)
    plt.title(f'SAE Feature-{attr_type.title()} Attribution Associations\n(Top {len(df_top)} significant)')
    plt.xlabel('Attributions')
    plt.ylabel('SAE Features')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

    return df_top

# Create heatmaps for each attribution type
for attr_type in ['continuous', 'categorical', 'binary']:
    print(f"\n=== {attr_type.upper()} ASSOCIATIONS ===")
    top_associations = create_association_heatmap(results, attr_type)

    if top_associations is not None and len(top_associations) > 0:
        print(f"Top 5 {attr_type} associations:")
        print(top_associations.nsmallest(5, 'q_value')[['attribution', 'feature', 'q_value', 'effect_size']])

### Linear Probe Results

In [ ]:
# Display linear probe performance
if 'multivariate' in results and 'linear_probes' in results['multivariate']:
    print("\n=== LINEAR PROBE RESULTS ===")

    probes = results['multivariate']['linear_probes']

    for attr_name, probe_result in probes.items():
        print(f"\n{attr_name}:")

        if 'auc' in probe_result:
            print(f"  AUC: {probe_result['auc']:.3f} ± {probe_result.get('auc_std', 0):.3f}")

        if 'r2' in probe_result:
            print(f"  R²: {probe_result['r2']:.3f} ± {probe_result.get('r2_std', 0):.3f}")

        if 'f1' in probe_result:
            print(f"  F1: {probe_result['f1']:.3f} ± {probe_result.get('f1_std', 0):.3f}")
else:
    print("\nNo linear probe results available")

## Step 7: Detailed Feature Investigation

Examine the most interesting SAE features in detail

In [ ]:
def analyze_top_features(results, top_n=5):
    """
    Analyze the most significant SAE features across all attributions.
    """
    all_results = []

    # Collect all significant results
    for attr_type in ['binary', 'categorical', 'continuous']:
        if attr_type in results['univariate']:
            for attr_name, attr_results in results['univariate'][attr_type].items():
                for result in attr_results:
                    if result.get('q_value', 1.0) < 0.05:
                        all_results.append({
                            'feature_idx': result['feature_idx'],
                            'attribution': attr_name,
                            'attr_type': attr_type,
                            'q_value': result['q_value'],
                            'effect_size': result.get('effect_size', 0),
                            'statistic': result.get('statistic', 0)
                        })

    if len(all_results) == 0:
        print("No significant associations found")
        return

    df_all = pd.DataFrame(all_results)

    # Find features with most associations
    feature_counts = df_all['feature_idx'].value_counts()
    top_features = feature_counts.head(top_n).index

    print(f"\n=== TOP {top_n} MOST ASSOCIATED SAE FEATURES ===")

    for feature_idx in top_features:
        feature_results = df_all[df_all['feature_idx'] == feature_idx]

        print(f"\nSAE Feature {feature_idx}:")
        print(f"  - {len(feature_results)} significant associations")
        print(f"  - Sparsity: {(sae_features[:, feature_idx] == 0).mean():.1%}")
        print(f"  - Mean activation: {sae_features[:, feature_idx].mean():.3f}")
        print(f"  - Std activation: {sae_features[:, feature_idx].std():.3f}")

        # Show top associations
        top_assoc = feature_results.nsmallest(3, 'q_value')
        print("  Top associations:")
        for _, row in top_assoc.iterrows():
            print(f"    - {row['attribution']} ({row['attr_type']}): q={row['q_value']:.2e}, effect={row['effect_size']:.3f}")

    return df_all, top_features

# Analyze top features
all_associations, top_feature_indices = analyze_top_features(results)

### Visualize Top Feature Activations

In [ ]:
if top_feature_indices is not None and len(top_feature_indices) > 0:
    # Plot activation patterns for top features
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()

    for i, feature_idx in enumerate(top_feature_indices[:6]):
        activations = sae_features[:, feature_idx]

        # Histogram of activations
        axes[i].hist(activations[activations > 0], bins=50, alpha=0.7)
        axes[i].set_title(f'SAE Feature {feature_idx}\nSparsity: {(activations == 0).mean():.1%}')
        axes[i].set_xlabel('Activation Value')
        axes[i].set_ylabel('Count')
        axes[i].set_yscale('log')

    plt.tight_layout()
    plt.suptitle('Activation Distributions for Top SAE Features', y=1.02)
    plt.show()

## Step 8: Save Results

Save the analysis results for further investigation

In [ ]:
# Save comprehensive results
output_dir = '../data/processed/sae_attribution_analysis/'
os.makedirs(output_dir, exist_ok=True)

# Save SAE features
np.save(f'{output_dir}/sae_features.npy', sae_features)
print(f"Saved SAE features to {output_dir}/sae_features.npy")

# Save analysis results as JSON
def make_json_serializable(obj):
    """Convert numpy types to Python types for JSON serialization."""
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {key: make_json_serializable(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [make_json_serializable(item) for item in obj]
    else:
        return obj

# Convert results to JSON-serializable format
json_results = make_json_serializable(results)

with open(f'{output_dir}/analysis_results.json', 'w') as f:
    json.dump(json_results, f, indent=2)
print(f"Saved analysis results to {output_dir}/analysis_results.json")

# Save summary statistics
if all_associations is not None:
    all_associations.to_csv(f'{output_dir}/all_associations.csv', index=False)
    print(f"Saved association summary to {output_dir}/all_associations.csv")

# Save metadata
adata.obs.to_csv(f'{output_dir}/cell_metadata.csv')
print(f"Saved cell metadata to {output_dir}/cell_metadata.csv")

print(f"\nAll results saved to: {output_dir}")

## Summary

This analysis pipeline:

1. ✅ **Data Collection**: Retrieved cells with geneformer embeddings from CellxGene Census
2. ✅ **SAE Processing**: Passed embeddings through trained sparse autoencoder
3. ✅ **Feature Extraction**: Extracted sparse latent features from SAE
4. ✅ **Attribution Analysis**: Comprehensive statistical analysis of feature-attribution correlations
5. ✅ **Results Investigation**: Identified top features and their biological associations

### Key Findings:
- **SAE Feature Dimensionality**: {sae_features.shape[1]} latent features
- **Overall Sparsity**: ~{(sae_features == 0).mean():.1%}
- **Significant Associations**: Multiple SAE features correlate with cell types, QC metrics, and other attributions
- **Linear Probes**: SAE features can predict various cell attributions with varying degrees of success

### Next Steps:
1. **Biological Interpretation**: Examine what genes/pathways the top SAE features capture
2. **Feature Refinement**: Optimize SAE architecture based on attribution correlations
3. **Validation**: Test findings on independent datasets
4. **Mechanistic Analysis**: Investigate how SAE features relate to biological processes